In [1]:
import pandas as pd
import os
import sys
parent_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(os.path.abspath(parent_path))
from constants import HAWAJEZ ,INFORMATION_WORDS,WORDS_SYNENOMS
from fuzzywuzzy import fuzz
import random

c:\Users\nasser\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
data = pd.read_csv('../data/cleaned_transformed_data.csv')

In [3]:
# make the dataframe from full_text column only
data = data[['full_text']]

In [4]:
data.shape

(23261, 1)

In [5]:
data.fillna('', inplace=True)

In [6]:
def replace_word_with_hawajez(text):
    """Replace a word in the given text with a random word from the Hawajez list."""
    for word in HAWAJEZ:
        if word in text:
            new_word = random.choice(HAWAJEZ)
            return text.replace(word, new_word)
    return text

In [7]:

def replace_status_with_new_status(text):
    """Replace a word in the given text with a random word from the Hawajez list."""
    for  info in INFORMATION_WORDS:
        if info in text:
            new_info = random.choice(INFORMATION_WORDS)
            return text.replace(info, new_info)
    return text

In [8]:
def replace_word_with_synonym(text):
    """Replace a word in the given text with a random word from the Hawajez list."""
    for word in WORDS_SYNENOMS:
        # use fuzzywuzzy to find the closest word instead of exact match

        score = fuzz.partial_ratio(word, text)
        
        if score > 80:
            new_word = random.choice(WORDS_SYNENOMS[word])
            text = text.replace(word, new_word)
    return text

In [9]:
import concurrent.futures
import warnings

warnings.filterwarnings('ignore')


new_data = pd.DataFrame(columns=["full_text"])
def process_row(row):
    # print flag every 1000 rows
    if row.name % 1000 == 0:
        print(row.name)
    new_rows = []
    for i in range(0, random.randint(10,20)):
        new_text = replace_word_with_hawajez(row["full_text"])
        new_text_status = replace_status_with_new_status(new_text)
        new_text_synonym = replace_word_with_synonym(new_text_status)
        if new_text:
            new_rows.append({"full_text": new_text_synonym})
    return new_rows

# Use ThreadPoolExecutor to run the loop in parallel threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit a thread for each row in the data
    future_to_row = {executor.submit(process_row, row): row for _, row in data.iterrows()}
    # Collect the results as they become available
    for future in concurrent.futures.as_completed(future_to_row):
        new_rows = future.result()
        # Append the new rows to the new data
        if new_rows:
            new_data = new_data.append(new_rows, ignore_index=True)

pd.set_option('display.max_colwidth', None)
new_data.shape
# should i concatinat the new data with the old data ? 
# a: yes
# show me how ?
data = pd.concat([data, new_data], ignore_index=True)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000


In [10]:
data.shape

(372085, 1)

In [11]:
# check if theres any null values
data.isnull().sum()

full_text    0
dtype: int64

In [12]:
# suffle the rows
data = data.sample(frac=1).reset_index(drop=True)

In [20]:
data.iloc[50000:50010]

,full_text
50000,يا يجماعة زعتره مسكره بس للخارج من الكنتينرا رامالله والكنتينرا قبلان
50001,حواره أزمة
50002,دوار المربعه اغلاقحركة سير قوية من كل الاتجاهات
50003,بيت فويك مسكر
50004,شباب الكنتينر زطمه ولا مسكرة
50005,قرني شمرون مازم سكرو
50006,حواره مغلق|||صحيحة الكونتينر أزمة ل الفايت و الي طالع على نابلس
50007,محسوم بيت فوريكء سالك وما في ازمة الكل طلع في اتجاه نابلس اكتضاض على الطالع
50008,في عملية اطلاق نار ع مسكر الكنتينر الرجاء الانتباه
50009,بيت فوريك تشديد عالخارج الان


In [285]:
data = data[['full_text']]
data.to_csv('../data/more_data_v4.csv', index=False)